In [100]:
# import os
# dir_name = 'Data'
# if not os.path.isdir('./' + dir_name + '/'):
#   os.mkdir('./' + dir_name + '/')
#   !unrar x 'data.rar' -d 'Data'

In [101]:
import os
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')

In [2]:
path_base = 'C:\Projects\Project1'
dir_name = 'inputs'
full_path = os.path.join('/',path_base,dir_name,'data.csv')

data = pd.read_csv(full_path)
data

,password,strength
0,kzde5577,1
1,kino3434,1
2,visi7k1yr,1
3,megzy123,1
4,lamborghin1,1
...,...,...
669874,10redtux10,1
669875,infrared1,1
669876,184520socram,1
669877,marken22a,1


In [3]:
data.isna().sum()

password    1
strength    0
dtype: int64

In [4]:
data = data.dropna(how = "any")

In [5]:
data['strength'].value_counts()

1                                496801
0                                 89701
2                                 83137
ckrguzel61@hotmail.com                2
igoubane@gmail.com                    2
                                  ...  
karaselim.colakoglu@gmail.com         1
kaankarayel_2012@hotmail.com          1
kudigaf-59@hotmail.com                1
selim_sarikaya42@hotmail.com          1
efsane.mardinli@gmail.com             1
Name: strength, Length: 239, dtype: int64

In [6]:
data = data.loc[(data.strength=='0') | (data.strength=='1') | (data.strength=='2'),:]
sum = data['strength'].value_counts().sum()
# data['strength'].value_counts()/sum
data['strength'].value_counts()

1    496801
0     89701
2     83137
Name: strength, dtype: int64

# We choose the same proportion of classes for the strength column.

In [7]:
strength0 = data.loc[(data.strength == '0')]
strength1 = data.loc[(data.strength == '1')]
strength2 = data.loc[(data.strength == '2')]

In [8]:
(strength0.shape, strength1.shape, strength2.shape)

((89701, 2), (496801, 2), (83137, 2))

In [9]:
strength0 = strength0.head(83137)
strength1 = strength1.head(83137)
strength2 = strength2.head(83137)

In [10]:
(strength0.shape, strength1.shape, strength2.shape)

((83137, 2), (83137, 2), (83137, 2))

In [11]:
data = strength0.append(strength1.append(strength2))

In [12]:
X = data.password
y = data.strength
(X.shape, y.shape) 


((249411,), (249411,))

# We separate each password adding white spaces to each character.

In [13]:
def word_to_characters(inputs):
  characters = []
  st = " "
  for i in inputs:
    characters.append(i)
  return st.join(characters)

In [14]:
X2 = X.map(lambda x: word_to_characters(x))
X2.head()

21      i n t e l 1
35    l s d l s d 1
44      o l m a z .
50      o k > b d k
58      d 0 4 m 1 1
Name: password, dtype: object

# We make some experiments with different tokenizers to evaluate which one shows better results.

In [ ]:
# from nltk.tokenize import TweetTokenizer
# tknzr = TweetTokenizer()
# s0 = "This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"
# tknzr.tokenize(s0)
# # ['This', 'is', 'a', 'cooool', '#dummysmiley', ':', ':-)', ':-P', '<3', 'and', 'some', 'arrows', '<', '>', '->', '<--']

In [ ]:
# from nltk.tokenize import RegexpTokenizer
# s = "Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.\n\nThanks."
# tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
# tokenizer.tokenize(s)
# # ['Good', 'muffins', 'cost', '$3.88', 'in', 'New', 'York', '.',
# # 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']

In [15]:
from nltk.tokenize import WhitespaceTokenizer
s = "A b c de"
WhitespaceTokenizer().tokenize(s)

['A', 'b', 'c', 'de']

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(tokenizer = word_to_characters)
# vectorizer = TfidfVectorizer(tokenizer = TweetTokenizer().tokenize)
# vectorizer = TfidfVectorizer()
vectorizer = TfidfVectorizer(tokenizer = WhitespaceTokenizer().tokenize)


In [103]:
os.chdir("C:\Projects\Project1\Models")

In [115]:
pickle.dump(vectorizer, open("vectorizer.pkl",'wb'))  
pickle.load(open("vectorizer.pkl",'rb')) 

TfidfVectorizer(tokenizer=<bound method RegexpTokenizer.tokenize of WhitespaceTokenizer(pattern='\\s+', gaps=True, discard_empty=True, flags=re.UNICODE|re.MULTILINE|re.DOTALL)>)

In [105]:
def load(filename_pkl):
    path_base = 'C:\Projects\Project1'
    dir_name = "Models"
    full_path = os.path.join('/',path_base,dir_name,filename_pkl)    
    # full_path = os.path.join('./',path_base,filename_pkl)
    pickle.load(open(full_path, 'rb'))
    # return print(full_path)

In [30]:
XX = vectorizer.fit_transform(X2)

In [31]:
# -->   stratify = y   <-- produces the same proportion as in X and y
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(XX, y, test_size = 0.2)

In [32]:
y_train_df = pd.DataFrame(y_train)
sum = y_train_df.value_counts().sum()
y_train_df.value_counts()/sum

strength
2           0.333863
1           0.333221
0           0.332916
dtype: float64

In [33]:
y_test_df = pd.DataFrame(y_test)
sum = y_test_df.value_counts().sum()
y_test_df.value_counts()/sum

strength
0           0.335004
1           0.333781
2           0.331215
dtype: float64

In [34]:
(x_train.shape , x_test.shape , y_train.shape , y_test.shape)

((199528, 137), (49883, 137), (199528,), (49883,))

# Model 1

In [121]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(penalty = 'l2', multi_class = 'ovr')
log_reg

LogisticRegression(multi_class='ovr')

In [122]:
log_reg.fit(x_train,y_train)

LogisticRegression(multi_class='ovr')

In [123]:
log_reg.score(x_train,y_train)

0.7957780361653503

In [124]:
log_reg.score(x_test, y_test)

0.7961830683800092

In [125]:
pickle.dump(log_reg, open("log_reg.pkl",'wb'))  
pickle.load(open("log_reg.pkl",'rb')) 

LogisticRegression(multi_class='ovr')

# We make a simple test


In [126]:
Password = pd.DataFrame(["Peztwurt169#", "ñzjmbjfufk", "ldcubmd589","Rutla100#"], columns=["A"])
Password

,A
0,Peztwurt169#
1,ñzjmbjfufk
2,ldcubmd589
3,Rutla100#


In [127]:
A2 = Password["A"].map(lambda x: word_to_characters(x))
A2

0    P e z t w u r t 1 6 9 #
1        ñ z j m b j f u f k
2        l d c u b m d 5 8 9
3          R u t l a 1 0 0 #
Name: A, dtype: object

In [128]:
Password["A"]

0    Peztwurt169#
1      ñzjmbjfufk
2      ldcubmd589
3       Rutla100#
Name: A, dtype: object

In [129]:
vectorizer_model = pickle.load(open("vectorizer.pkl",'rb')) 

Password_vec = vectorizer_model.transform(A2)

In [130]:
log_reg_Model = pickle.load(open("log_reg.pkl",'rb')) 
log_reg_Model

LogisticRegression(multi_class='ovr')

In [131]:
log_reg_Model.predict(Password_vec)

array(['2', '0', '1', '2'], dtype=object)

# Model 2

In [132]:
from sklearn.naive_bayes import BernoulliNB
Bernoulli_Naive_Bayes = BernoulliNB()
Bernoulli_Naive_Bayes.fit(x_train,y_train)

BernoulliNB()

In [133]:
Bernoulli_Naive_Bayes.score(x_train,y_train)

0.8198348101519586

In [134]:
Bernoulli_Naive_Bayes.score(x_test,y_test)

0.819136780065353

In [136]:
pickle.dump(Bernoulli_Naive_Bayes, open("Bernoulli_Naive_Bayes.pkl",'wb'))
pickle.load(open("Bernoulli_Naive_Bayes.pkl",'rb')) 

BernoulliNB()

# Model 3

In [137]:
from sklearn.tree import DecisionTreeClassifier
Decision_Tree = DecisionTreeClassifier()
Decision_Tree.fit(x_train,y_train)

DecisionTreeClassifier()

In [138]:
Decision_Tree.score(x_train,y_train)

0.9998095505392727

In [139]:
Decision_Tree.score(x_test,y_test)

0.8861335525128802

In [140]:
pickle.dump(Decision_Tree, open("Decision_Tree.pkl",'wb'))
pickle.load(open("Decision_Tree.pkl",'rb')) 

DecisionTreeClassifier()

# Model 4

In [149]:
from sklearn.ensemble import RandomForestClassifier
Random_Forest = RandomForestClassifier(n_estimators=100, max_depth=50, criterion='entropy')
Random_Forest.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=50)

In [150]:
Random_Forest.score(x_train,y_train)

0.9998095505392727

In [151]:
Random_Forest.score(x_test,y_test)

0.9468155483832167

In [152]:
pickle.dump(Random_Forest, open("Random_Forest.pkl",'wb'))
pickle.load(open("Random_Forest.pkl",'rb')) 

RandomForestClassifier(criterion='entropy', max_depth=50)

# Model 5

In [157]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver = 'adam', alpha = 1e-5, max_iter=400, activation='logistic')

In [158]:
mlp.fit(x_train,y_train)

MLPClassifier(activation='logistic', alpha=1e-05, max_iter=400)

In [159]:
mlp.score(x_train,y_train)

MemoryError: Unable to allocate 152. MiB for an array with shape (199528, 100) and data type float64

In [160]:
mlp.score(x_test,y_test)

0.9730569532706533

In [161]:
pickle.dump(mlp, open("multi_layer_perceptron.pkl",'wb'))
pickle.load(open("multi_layer_perceptron.pkl",'rb')) 


MLPClassifier(activation='logistic', alpha=1e-05, max_iter=400)